In [162]:
import pandas as pd
import missingno as msno
from datetime import datetime
import numpy as np
import sklearn.compose._column_transformer
import seaborn as sns
import matplotlib.pyplot as plt

In [163]:
customers = pd.read_csv("../data/olist_customers_dataset.csv") # Client
geolocalisation = pd.read_csv("../data/olist_geolocation_dataset.csv") # Loc

order_items = pd.read_csv("../data/olist_order_items_dataset.csv") # Commandes items
order_payments = pd.read_csv("../data/olist_order_payments_dataset.csv") # Commandes paiement
order_reviews = pd.read_csv("../data/olist_order_reviews_dataset.csv") # Commandes reviews
orders = pd.read_csv("../data/olist_orders_dataset.csv") # Commandes

sellers = pd.read_csv("../data/olist_sellers_dataset.csv") # Vendeurs
products = pd.read_csv("../data/olist_products_dataset.csv") # Produits
cat_trans = pd.read_csv('../data/product_category_name_translation.csv') # Translate


In [164]:
# orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
# orders.set_index('order_approved_at', inplace=True)

In [165]:
order_cust = pd.merge(left=customers, right=orders, how="left", left_on="customer_id", right_on="customer_id")
forder_cust = pd.merge(left=order_cust, right=order_items, how="outer", left_on="order_id", right_on="order_id")

forder_cust['total'] = forder_cust[["price", "freight_value"]].sum(axis=1)

## Recence 

In [166]:
dates = ['order_purchase_timestamp', 'order_approved_at','order_delivered_carrier_date', 'order_delivered_customer_date','order_estimated_delivery_date','shipping_limit_date']
forder_cust[dates] =forder_cust[dates].apply(pd.to_datetime, format='%Y-%m-%d')
date_1 = forder_cust.sort_values(by ='order_approved_at',ascending = False)


recence = date_1.groupby('customer_unique_id').agg(last_date =('order_approved_at','max')).reset_index()


recence['periode']=(date_1.order_approved_at.max()-recence.last_date).dt.days
recence.dropna(inplace=True)
recence.periode = recence.periode.astype(int)

def recence_transform(row):
    if row['periode'] <= 140:
        return 1
    if row['periode']> 140 and row['periode'] <= 280:
        return 2
    if row['periode']>280 and row['periode'] <= 420:
        return 3
    if row['periode']> 420 and row['periode'] <= 560:
        return 4
    if row['periode']> 560:
        return 5 

recence['r_value']=recence.apply(lambda row: recence_transform(row),axis=1)
recence.head()

,customer_unique_id,last_date,periode,r_value
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 11:11:18,116,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 18:25:44,118,1
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,541,4
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:49:17,325,3
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 20:06:52,292,3


In [167]:

liste1 =cat_trans.product_category_name.tolist()
liste2 = cat_trans.product_category_name_english.tolist()
dict = {}
for key in liste1:
    for value in liste2:
        dict[key] = value
        liste2.remove(value)
        break
products.product_category_name=products.product_category_name.map(dict)
order_reviews.drop(['review_comment_title','review_comment_message'],inplace=True,axis=1)
order_reviews['review_score'].dropna(inplace=True)

## Merging pour l'analyse des commandes

### reajustement du df payments pour obtenir une ligne = une commande

In [168]:
p1 =order_payments.groupby('order_id').agg(total_pay =('payment_value','sum')).reset_index()


In [169]:
p1 = order_payments.groupby(['order_id','payment_installments']).agg(payment_type = ('payment_type',apply(lambda)),total_pay =('payment_value','sum')).reset_index()
p2=p1.groupby('order_id')['payment_installments'].apply(list).reset_index()
p3 = p1.groupby('order_id')['payment_type'].apply(list).reset_index()

In [170]:
p3.payment_type= p3.payment_type.astype(str)

In [171]:
order_payments[order_payments.order_id=='02d0cd05f04ab2e3c1acc473b0abc865']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
79945,02d0cd05f04ab2e3c1acc473b0abc865,2,credit_card,2,29.64
99441,02d0cd05f04ab2e3c1acc473b0abc865,1,credit_card,3,56.40


In [172]:
p3[p3.payment_type == "['credit_card', 'credit_card']"]

,order_id,payment_type
1080,02d0cd05f04ab2e3c1acc473b0abc865,"['credit_card', 'credit_card']"
2255,05bd4a1f3930d20f351ce8c406d77103,"['credit_card', 'credit_card']"
2307,05d885b852158b0280c4abe2cd5108b4,"['credit_card', 'credit_card']"
2847,0741ee0ba3cb46d86325be2879e18706,"['credit_card', 'credit_card']"
2888,0758aeea44cefb9c7f59333c7c691714,"['credit_card', 'credit_card']"
...,...,...
95838,f6cbd0bea76d160391128a43b629ae9f,"['credit_card', 'credit_card']"
96389,f8303e6eff198d1749d04649a1eb2bb6,"['credit_card', 'credit_card']"
98514,fd87d98feff1673e27f7302bceb00cf6,"['credit_card', 'credit_card']"
98976,febdd4292b7bc064ae8e10f572aa2ce9,"['credit_card', 'credit_card']"


### Merging products and order_items et les regrouper par commandes uniques

In [173]:
order_items.drop(['seller_id','price','freight_value'],inplace=True,axis=1)


In [174]:
products_item = pd.merge(order_items,products,how ='left',left_on='product_id',right_on='product_id')
a = products_item.groupby(['order_id','product_category_name']).agg(total_commande=('order_item_id','count'),prod =('product_id','nunique')).reset_index()
c = a.groupby('order_id').agg(items_commande = ('total_commande','sum'),prod_commande=('prod','sum')).reset_index()
b = a.groupby('order_id')['product_category_name'].apply(list).reset_index()


In [175]:
order_items.order_item_id.value_counts()

1     98666
2      9803
3      2287
4       965
5       460
6       256
7        58
8        36
9        28
10       25
11       17
12       13
13        8
14        7
15        5
16        3
17        3
18        3
19        3
20        3
21        1
Name: order_item_id, dtype: int64

In [190]:
new_products_items = pd.merge(c,b,how='left',left_on='order_id',right_on='order_id')
new_products_items[new_products_items.order_id=='02d0cd05f04ab2e3c1acc473b0abc865']

,order_id,items_commande,prod_commande,product_category_name
1052,02d0cd05f04ab2e3c1acc473b0abc865,1,1,[fashion_bags_accessories]


In [194]:
p1[p1.order_id=='02d0cd05f04ab2e3c1acc473b0abc865']

,order_id,payment_installments,payment_type,total_pay
1091,02d0cd05f04ab2e3c1acc473b0abc865,2,credit_card,29.64
1092,02d0cd05f04ab2e3c1acc473b0abc865,3,credit_card,56.40


In [191]:
p3[p3.order_id=='02d0cd05f04ab2e3c1acc473b0abc865']

,order_id,payment_type
1080,02d0cd05f04ab2e3c1acc473b0abc865,"['credit_card', 'credit_card']"


In [192]:
order_payments[order_payments.order_id=='02d0cd05f04ab2e3c1acc473b0abc865']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
79945,02d0cd05f04ab2e3c1acc473b0abc865,2,credit_card,2,29.64
99441,02d0cd05f04ab2e3c1acc473b0abc865,1,credit_card,3,56.40


In [193]:
products_item[products_item.order_id == '02d0cd05f04ab2e3c1acc473b0abc865']

,order_id,order_item_id,product_id,shipping_limit_date,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1212,02d0cd05f04ab2e3c1acc473b0abc865,1,77fa3381855b6763c52ca3b72824f508,2018-03-29 14:30:49,fashion_bags_accessories,60.0,177.0,6.0,250.0,16.0,2.0,20.0


### ok

In [179]:
new_order_items = order_items.groupby('order_id').agg(total_commande=('order_item_id','nunique'),prod =('product_id','nunique')).reset_index()



In [180]:
p1[p1.order_id=='005d9a5423d47281ac463a968b3936fb']

,order_id,payment_installments,payment_type,total_pay
136,005d9a5423d47281ac463a968b3936fb,10,credit_card,145.26


In [181]:
products[products.product_id =='8338cef8355d238f43711dcb9c0657b2']

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
4642,8338cef8355d238f43711dcb9c0657b2,furniture_decor,59.0,473.0,1.0,700.0,30.0,3.0,20.0


In [182]:
order_items[order_items.order_id =='005d9a5423d47281ac463a968b3936fb']

,order_id,order_item_id,product_id,shipping_limit_date
151,005d9a5423d47281ac463a968b3936fb,1,fb7a100ec8c7b34f60cec22b1a9a10e0,2017-10-24 12:28:16
152,005d9a5423d47281ac463a968b3936fb,2,4c3ae5db49258df0784827bdacf3b396,2017-10-24 12:28:16
153,005d9a5423d47281ac463a968b3936fb,3,4c3ae5db49258df0784827bdacf3b396,2017-10-24 12:28:16


In [183]:
new_order_items[new_order_items['prod']==3]

,order_id,total_commande,prod
447,012a238ab54294a3b365812ccc82b135,3,3
836,023669233121f0fb7899e5be2b22885f,3,3
1259,034afd1b074e35a1d1dcf70c4a5641b5,3,3
1607,0420dbc50fc554e303e4b2d6b39063f6,3,3
1775,048fec87f5858e3a91ce55d3a944b457,3,3
...,...,...,...
98255,fee26dd998861d207782234217b367ec,3,3
98297,fef9630717f341d93127dbf1d27650c9,3,3
98308,ff00a56fe9475a175cd651d77c707a09,3,3
98563,ffb8f7de8940249a3221252818937ecb,3,3


In [184]:
prorder_items = pd.merge(left=df,right=orders,how='left',left_on='order_id',right_on='order_id') #orders_items,orders,products
merged_review = pd.merge(left=prorder_items,right=order_reviews,how='inner',left_on='order_id',right_on='order_id')#review
merged_payment = pd.merge(left=merged_review,right=order_payments,how='inner',left_on='order_id',right_on='order_id')#payments
merged_customers = pd.merge(left=merged_payment,right=customers,how='inner',left_on='customer_id',right_on='customer_id')#rcustomers
merged_sellers = pd.merge(left=merged_customers ,right=sellers,how='inner',left_on='seller_id',right_on='seller_id')#sellers

In [185]:
merged_sellers.drop_duplicates(inplace=True)

In [186]:
merged_sellers.drop(['review_id','customer_id','product_height_cm','product_width_cm','product_weight_g','product_length_cm','review_creation_date'],axis=1,inplace=True)

In [187]:
new = merged_sellers.groupby('order_id').agg(total_items = ('order_item_id','count'),total_price = ('price','sum'),total_freight=('freight_value','sum')).reset_index()



In [188]:
n_dims = X_train_standard.shape[1]
model = PCA(n_components=n_dims)
model.fit(X_train_standard)

variances = model.explained_variance_ratio_

meilleur_dims = np.argmax(np.cumsum(variances) > 0.90)

import matplotlib.pyplot as plt
plt.bar(range(n_dims), np.cumsum(variances))
plt.hlines(0.90, 0, meilleur_dims, colors='r')
plt.vlines(meilleur_dims, 0, 0.90, colors='r')

NameError: name 'X_train_standard' is not defined